## TP2 Filtros
## Filtro Gaussiano y Fourier

In [ ]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib inline
#%matplotlib

# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV 
import cv2 as cv
# Usamos las poderosas herramientas de graficación de matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt
#Usamos también las poderosas herramientas de procesamiento de imágenes en python de Scikit
#from skimage.util import random_noise
# No pude usar esta libreria. Intente descargarla y actualizar las plataformas. Aun asi no puede. Trabaje en Windows. Quizas en Ubuntu si sea posible

# Importamos librerías para manejo de tiempo
import time

#### Filtro Gaussiano

In [ ]:
# Para crear un kernel Gaussiano (unidimensional)
nucleo_g = cv.getGaussianKernel(ksize=5, sigma=1.5) 
# Parámetros:
# ksize - Tamaño del núcleo, positivo impar (3,5,...)
# sigma - Desvío estándar de la Gaussiana. Si no es positivo, se calcula como sigma = 0.3*((ksize-1)*0.5 - 1) + 0.8 
# ktype - Tipo de coeficientes del filtro (Optional). Puede ser CV_32F o CV_64F

print(nucleo_g)

In [ ]:
# Suavizado Gaussiano usando el Kernel
#=====================================
img_orig = cv.imread('Black_Panther.jpg')
 
# Creamos un núcleo Gaussiano unidimensional
kX = cv.getGaussianKernel(47,15)
kY = cv.getGaussianKernel(15,22)
 
# Aplicamos el filtro Gaussiano. Con distinto núcleo para X e Y!
b = cv.sepFilter2D(img_orig, -1, kX, kY)

# Más fácil utilizando una única línea
# (cuando el kernel es cuadrado)
#=====================================
c = cv.GaussianBlur(img_orig, (37,37), 10)

# Mostramos la imagen
out1 = cv.hconcat([img_orig,b])
out2 = cv.hconcat([img_orig,c])

# Tengo problemas con las pantallas emergentes
plt.figure()
plt.imshow(out1)
plt.title ('Black_Panther suave 1')
plt.show()

plt.figure()
plt.imshow(out2)
plt.title ('Black_Panther suave 2')
plt.show()


In [ ]:
from random import gauss

# funciones auxiliares
def fft2_mod(image):

  shifted_image_fft = np.fft.fftshift(np.fft.fft2(image))
  return 20*np.log(np.abs(shifted_image_fft))

def create_gauss_filter(h, w, k_size, sigma):

    #ker, ker_tf = create_gauss_filter(h, w, k_size, sigma)
    # Suavizado Gaussiano usando el Kernel
    #=====================================
    #img_orig = cv.imread('Black_Panther.jpg')
    
    # Creamos un núcleo Gaussiano unidimensional
    #kX = cv.getGaussianKernel(47,15)
    #kY = cv.getGaussianKernel(15,22)
    
    # Aplicamos el filtro Gaussiano. Con distinto núcleo para X e Y!
    #b = cv.sepFilter2D(img_orig, -1, kX, kY)

    # Más fácil utilizando una única línea
    # (cuando el kernel es cuadrado)
    #=====================================
    ker = cv.getGaussianKernel(k_size, sigma) 
    
    #ker_tf = fft2_mod(ker)
    ker_tf = np.fft.fft2(ker)
    ker_tf = np.abs(ker_tf)
    # Mostramos la imagen
    #out1 = cv.hconcat([img,b])
    #out2 = cv.hconcat([img,c])

    # Tengo problemas con las pantallas emergentes
    #plt.figure()
    #plt.imshow(out1)
    #plt.title ('Black_Panther suave 1')
    #plt.show()


    return ker, ker_tf

In [ ]:
# leo la imagen en escala de grises
img = cv.imread('Black_Panther.jpg', cv.IMREAD_GRAYSCALE)
# obtengo el modulo de la transformada de fourier
#mod = fft2_mod(img)

h, w = img.shape
#lp_filter = create_lp_filter(h, w, radius=40)
#hp_filter = create_hp_filter(h, w, radius=20)


ker, ker_tf = create_gauss_filter(h, w, k_size=15, sigma=15)
#ker = create_gauss_filter(h, w, k_size=5, sigma=5)
#ker_tf = create_gauss_filter(h, w, k_size=5, sigma=5)

plt.figure(figsize=(12,12))
plt.subplot(1,2,1)
plt.axis('off')
plt.title("Kernel Gausiano")
plt.imshow(ker, cmap='jet')
plt.subplot(1,2,2)
plt.axis('off')
plt.title("FFT del kernel")
plt.imshow(np.abs(ker_tf), cmap='jet')
#plt.imshow(ker_tf, cmap='jet')

In [ ]:

#ker_tf = cv.GaussianBlur(img, ker, 15)
ker_tf = cv.GaussianBlur(img, (15,15), 5)
#img_fft = fft2_mod(img)
img_fft = np.fft.fft2(img)
filtered = img_fft * ker_tf

img_gauss = np.real(np.fft.ifft2(np.fft.ifftshift(filtered)))


img_gauss = np.fft.ifftshift(np.real(np.fft.ifft2(filtered)))
plt.figure(figsize=(8,8))
plt.imshow(img_gauss, cmap='gray')
plt.title('Desenfoque gausiano')

In [ ]:
# Cargo la imagen
#================
img = cv.imread('metalgrid.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Aplico Canny
#=============
start = time.time()
edges = cv.Canny(img,40,105,L2gradient=True)
elapsed = time.time()-start
print('Tiempo de procesamiento {} segundos'.format(elapsed))

# Muestro la imagen
#==================
cv.imshow("CannyCV",edges)
plt.figure()
plt.imshow(edges)
plt.title ('CannyCV')
plt.show()


# Aplicamos Sobelx en 'float32' y luego convertimos de nuevo a 8-bit para evitar overflow
sobelx_64 = cv.Sobel(edges,cv.CV_32F,1,0,ksize=3)
absx_64 = np.absolute(sobelx_64)
sobelx_8u1 = absx_64/absx_64.max()*255
sobelx_8u = np.uint8(sobelx_8u1)

# De igual modo para Sobely
sobely_64 = cv.Sobel(edges,cv.CV_32F,0,1,ksize=3)
absy_64 = np.absolute(sobely_64)
sobely_8u1 = absy_64/absy_64.max()*255
sobely_8u = np.uint8(sobely_8u1)

# De los gradiente calculamos Magnitud y lo pasamos a 8-bit (Opcional)
mag = np.hypot(sobelx_8u, sobely_8u)
mag = mag/mag.max()*255
mag = np.uint8(mag)


# Encontramos la Dirección y la pasamos a grados
theta = np.arctan2(sobely_64, sobelx_64)
angle = np.rad2deg(theta)


cv.imshow("Magnitud de gradiente",mag)
plt.figure()
plt.imshow(mag)
plt.title ('Magnitud de gradiente')
plt.show()

cv.imshow("Dirección de gradiente",angle)
plt.figure()
plt.imshow(angle)
plt.title ('Dirección de gradiente')
plt.show()